In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_chroma import Chroma
from langchain_core.embeddings import Embeddings
from uuid import uuid4
import chromadb

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END

from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage, trim_messages, RemoveMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph.message import add_messages

from typing import Sequence
from typing_extensions import Annotated, TypedDict

from typing import List
from typing_extensions import TypedDict

import time

import cohere  

from langchain_cohere import ChatCohere
import getpass
import os
import json

with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
model = ChatCohere(cohere_api_key=api_key)

messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

embeddings = CohereEmbeddings(cohere_api_key=api_key, model="embed-english-v3.0", user_agent='langchain')
vector_store = Chroma(
    collection_name=f"test_history",
    embedding_function=embeddings,
    persist_directory=f"test_vectordbs",
)

retriever = vector_store.as_retriever(
    search_kwargs={'k': 1}
)

c:\Users\ryanz\Desktop\LLM-Based-NPC\.venv\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [2]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [3]:
current_time_id = int(time.time())
long_term_memory = []
metadata = []

for m in messages:
    current_time_id = int(time.time() * 1000)
    
    if isinstance(m, HumanMessage):
        entry = m.content
        long_term_memory.append(entry)
        metadata.append({"type": "HumanMessage", "timestamp": current_time_id})
    elif isinstance(m, AIMessage):
        entry = m.content
        long_term_memory.append(entry)
        metadata.append({"type": "AIMessage", "timestamp": current_time_id})
    
    time.sleep(0.01)


vector_store.add_texts(long_term_memory, metadatas=metadata)

['7f9b9103-cadc-402a-ab8a-c6de8f8d964b',
 '891cbaf6-f2bf-46de-bf49-fc2231615536',
 'e09ca585-1154-4247-a95c-46c466e6aee8',
 'a350845c-dffe-4b7c-b847-df3b5939a443',
 '97d05880-d069-42f1-8945-fd1b17c475a3',
 '16835cca-ae4a-484b-93ca-9dab4291c542',
 'ae034116-7246-475a-9d18-32f90a4fdb39',
 '35fb59a7-8181-4b4c-9b4c-616b0edbf4d1',
 '773b0429-80e2-4330-93df-56592f3523a3',
 '63ba50a8-0455-42c2-9697-00d358e82ea8']

In [4]:
rag_query = "hi"
print(f"Rag Query: {rag_query}")
documents = retriever.invoke(rag_query)
for res in documents:
    print(f"{res.page_content}")
    print(f"{res.metadata}")

Rag Query: hi
hi!
{'timestamp': 1729590558048, 'type': 'AIMessage'}


In [5]:
rag_query = "bob"
print(f"Rag Query: {rag_query}")
documents = retriever.invoke(rag_query)
for res in documents:
    print(f"{res.page_content}")
    print(f"{res.metadata}")

Rag Query: bob
hi! I'm bob
{'timestamp': 1729590558026, 'type': 'HumanMessage'}


In [6]:
metadata = documents[0].metadata
timestamp = metadata.get('timestamp', 'Timestamp not available')
print(f"Timestamp: {timestamp}")

Timestamp: 1729590558026


In [10]:
def query_within_time_frame(vector_store, timestamp, minutes=5):
    # Convert minutes to milliseconds
    time_delta_ms = minutes * 60 * 1000  # 5 minutes in milliseconds
    
    # Calculate the lower and upper bounds
    lower_bound = timestamp - time_delta_ms
    upper_bound = timestamp + time_delta_ms
    
    # Construct the query
    query = {
        "$and": [
            {"timestamp": {"$gte": lower_bound}},  # Greater than or equal to lower bound
            {"timestamp": {"$lte": upper_bound}}   # Less than or equal to upper bound
        ]
    }
    
    # Retrieve documents within the time frame
    documents = vector_store.get(where=query)
    
    return documents

In [15]:
result = query_within_time_frame(vector_store, timestamp)
result

combined = [(doc, metadata) for doc, metadata in zip(result["documents"], result["metadatas"])]
sorted_combined = sorted(combined, key=lambda x: x[1]["timestamp"])
for doc, metadata in sorted_combined:
    print(f"Document Content: {doc}")
    print(f"Timestamp: {metadata['timestamp']}")

Document Content: hi! I'm bob
Timestamp: 1729590558026
Document Content: hi!
Timestamp: 1729590558048
Document Content: I like vanilla ice cream
Timestamp: 1729590558063
Document Content: nice
Timestamp: 1729590558078
Document Content: whats 2 + 2
Timestamp: 1729590558093
Document Content: 4
Timestamp: 1729590558109
Document Content: thanks
Timestamp: 1729590558125
Document Content: no problem!
Timestamp: 1729590558140
Document Content: having fun?
Timestamp: 1729590558155
Document Content: yes!
Timestamp: 1729590558170


In [16]:
messages = []

for doc, metadata in sorted_combined:
    if metadata["type"] == "HumanMessage":
        messages.append(HumanMessage(content=doc))
    elif metadata["type"] == "AIMessage":
        messages.append(AIMessage(content=doc))

# Print the messages for verification
for message in messages:
    print(f"{message.content}")

hi! I'm bob
hi!
I like vanilla ice cream
nice
whats 2 + 2
4
thanks
no problem!
having fun?
yes!


In [17]:
messages

[HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [9]:
levelup = [15, 19, 24, 30, 37, 45, 54, 64, 75, 87, 100, 114, 129, 145, 162, 180, 199, 219, 240, 262, 285, 309, 334, 360, 387, 415, 444, 474, 505, 537, 570, 604, 639, 675, 712, 750, 789, 829, 870, 912, 955, 999, 1044, 1090, 1137, 1185, 1234, 1284, 1335, 1387, 1440, 1494, 1549, 1605, 1662, 1720, 1779, 1839, 1900, 1962, 2025, 2089, 2154, 2220, 2287, 2355, 2424, 2494, 2565, 2637, 2710, 2784, 2859, 2935, 3012, 3090, 3169, 3249, 3330, 3412, 3495, 3579, 3664, 3750, 3837, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000]

len(levelup)

while len(levelup) < 1000:
    levelup.append(5000)



print(", ".join(map(str, levelup)))
levelup

15, 19, 24, 30, 37, 45, 54, 64, 75, 87, 100, 114, 129, 145, 162, 180, 199, 219, 240, 262, 285, 309, 334, 360, 387, 415, 444, 474, 505, 537, 570, 604, 639, 675, 712, 750, 789, 829, 870, 912, 955, 999, 1044, 1090, 1137, 1185, 1234, 1284, 1335, 1387, 1440, 1494, 1549, 1605, 1662, 1720, 1779, 1839, 1900, 1962, 2025, 2089, 2154, 2220, 2287, 2355, 2424, 2494, 2565, 2637, 2710, 2784, 2859, 2935, 3012, 3090, 3169, 3249, 3330, 3412, 3495, 3579, 3664, 3750, 3837, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 50

[15,
 19,
 24,
 30,
 37,
 45,
 54,
 64,
 75,
 87,
 100,
 114,
 129,
 145,
 162,
 180,
 199,
 219,
 240,
 262,
 285,
 309,
 334,
 360,
 387,
 415,
 444,
 474,
 505,
 537,
 570,
 604,
 639,
 675,
 712,
 750,
 789,
 829,
 870,
 912,
 955,
 999,
 1044,
 1090,
 1137,
 1185,
 1234,
 1284,
 1335,
 1387,
 1440,
 1494,
 1549,
 1605,
 1662,
 1720,
 1779,
 1839,
 1900,
 1962,
 2025,
 2089,
 2154,
 2220,
 2287,
 2355,
 2424,
 2494,
 2565,
 2637,
 2710,
 2784,
 2859,
 2935,
 3012,
 3090,
 3169,
 3249,
 3330,
 3412,
 3495,
 3579,
 3664,
 3750,
 3837,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 4000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5